In [2]:
import pandas as pd
import seaborn as sns
import traintools
from trainconstants import *
from collections import defaultdict
import networkx as nx

sns.set()
%matplotlib inline

df = pd.read_csv("datasets/nsdata1.txt",  delimiter=r"\s+")
df['departure_time'] = df['departure_time'].map(traintools.to_minutes_past_midnight)
df['arrival_time'] = df['arrival_time'].map(traintools.to_minutes_past_midnight)
df.head()

,train_number,start,end,departure_time,arrival_time,first_class,second_class
0,2123,2,3,420,460,4,58
1,2123,3,4,463,518,14,328
2,2127,1,2,408,475,47,340
3,2127,2,3,481,521,35,272
4,2127,3,4,523,578,19,181


In [20]:
schedule_network= df.loc[:,['first_class', 'second_class']]
schedule_network['start'] = list(zip(df.start, df.departure_time))
schedule_network['end'] = list(zip(df.end, df.arrival_time))

G = nx.from_pandas_edgelist(df=schedule_network, source='start', target='end', edge_attr=['first_class', 'second_class'], create_using=nx.DiGraph)

In [21]:
nodes_per_station = defaultdict(list)

for node in G.nodes:
    nodes_per_station[node[0]].append(node)

for station in nodes_per_station.keys():
    nodes_per_station[station].sort()

    for index, stop in enumerate(nodes_per_station[station][:-1]):
        G.add_edge(stop, nodes_per_station[station][index + 1], edge_attr=(0, 0))

In [24]:
nx.dag_longest_path(G, weight='second_class')

[(4, 330),
 (4, 414),
 (3, 468),
 (3, 472),
 (2, 512),
 (2, 515),
 (2, 538),
 (2, 542),
 (3, 581),
 (3, 583),
 (3, 590),
 (3, 593),
 (2, 632),
 (2, 634),
 (1, 698),
 (1, 716),
 (2, 778),
 (2, 782),
 (3, 821),
 (3, 823),
 (3, 830),
 (3, 833),
 (2, 872),
 (2, 875),
 (1, 938),
 (1, 956),
 (2, 1018),
 (2, 1021),
 (3, 1063),
 (3, 1065),
 (3, 1070),
 (3, 1073),
 (2, 1112),
 (2, 1114),
 (2, 1138),
 (2, 1142),
 (3, 1181),
 (3, 1183),
 (3, 1190),
 (3, 1193),
 (2, 1232),
 (2, 1235),
 (1, 1298),
 (1, 1316),
 (2, 1378),
 (2, 1382),
 (3, 1434)]